In [ ]:
import pandas as pd
import datetime
from datetime import datetime
from datetime import timedelta

# Define the important variables here
target_fips = 6049
target_county_name = "Modoc"
target_data_file_name = f"Datasets/merged_tp_precip_wind_fmc_{target_county_name}.csv"

referral_fips = 6035
referral_country_name = "Lassen"

# Load the target county data
fire_data = pd.read_csv(target_data_file_name)
print(f"The dataset {target_data_file_name} contains (row, column) = ")
print(fire_data.shape)

# Initial the wind csv
wind = pd.read_csv("Datasets/wind_with_fips.csv")

referral_count = wind['county'].str.contains(referral_country_name).sum()
print(f"The referral county {referral_country_name} has {referral_count} rows of data in the wind_with_fips.csv")
    
# If not data found, exit the program
if ( int(referral_count) <= 0):
    quit()
    
# Extract the whole volume 
def filter_dataframe_by_value(df, column_name, value_to_find):
    filtered_rows = df[df[column_name] == value_to_find]
    return filtered_rows

if __name__ == "__main__":

    # Create a new DataFrame containing only rows with the specific value
    wind_referral = filter_dataframe_by_value(wind, 'fips', referral_fips)

# Add the new column here
def merge_dataframes_on_match(df1, df2, df1_column1, df2_column1, df2_column_to_sum1, df2_column_to_sum2):
    for index1, row1 in df1.iterrows():
        for index2, row2 in df2.iterrows():
            row1_datetime = datetime.strptime(str(row1[df1_column1]),  "%Y%m%d")
            row2_datetime = datetime.strptime(row2[df2_column1], "%Y-%m-%d")
            delta_days = (row1_datetime - row2_datetime).days
            print(f"Found a row with diff days {delta_days} between {row1_datetime} and {row2_datetime}")
            if delta_days < 50:
                print(f"Add data to the original : {row2[df2_column_to_sum1]}")
                df1.at[index1, df2_column_to_sum1] = row2[df2_column_to_sum1]
                #df1.at[index1, df2_column_to_sum2] = row2[df2_column_to_sum2]
                break

merge_dataframes_on_match(fire_data, wind_referral.sort_values("date"), 'DATE', 'date', 'aws', 'prcp')

# Save the data to its original file
fire_data.to_csv(target_data_file_name)
